# MusicGen-Large Finetuning Loop (Sequential Processing) - Transformers Version

このノートブックは、大規模なデータセットをZIPファイル単位で順次処理（解凍→学習→削除）しながらMusicGen-Largeをファインチューニングします。
Hugging Face Transformersライブラリを使用します。

## 前提条件
1. Google Driveに以下のデータがあること
   - `MyData/Archive_wavs/metadata.jsonl`: 全データのメタデータ
   - `MyData/Archive_wavs/archive_batch_xxxx.zip`: 音声データのZIPファイル群
2. A100 GPU推奨（VRAM容量のため）
3. **WandB API Key**: Colabのシークレット（鍵マーク）に `WANDB_API_KEY` という名前で登録してください。

In [ ]:
# @title 1. 環境設定とライブラリインストール
import os
import subprocess
import sys

print("潜在的な問題のあるライブラリをアンインストール中...")
# 関連するライブラリを一度アンインストール
!pip uninstall -y torch torchvision torchaudio torchcodec fastai sentence-transformers
print("アンインストール完了。")

print("ライブラリをインストール中...")

# CUDA 12.6対応のPyTorch (Nightly or Pre-release)
# 注意: ユーザー指定によりCUDA 12.6をターゲットにします。
!pip install --pre torch torchvision torchaudio torchcodec --index-url https://download.pytorch.org/whl/nightly/cu126

# Hugging Face Libraries & WandB
!pip install -U git+https://github.com/huggingface/transformers.git
!pip install -U datasets accelerate bitsandbytes wandb

# FFmpegのインストール
!apt-get update
!apt-get install -y ffmpeg
print("FFmpegのインストール完了。")

print("インストール完了。")

In [ ]:
# @title 1.1 ライブラリのインポート
import os
import sys
import subprocess
import shutil
import glob
import json
from pathlib import Path

import torch
import torchaudio
import numpy as np
from google.colab import drive, userdata
import wandb
from transformers import AutoProcessor, MusicgenForConditionalGeneration, Trainer, TrainingArguments, BitsAndBytesConfig
from datasets import load_dataset, Audio
from peft import LoraConfig, get_peft_model
import torch.cuda.amp as amp

print("ライブラリのインポート完了。")

In [ ]:
# @title 1.5 WandB ログイン
try:
    wandb_api_key = userdata.get('WANDB_API_KEY')
    wandb.login(key=wandb_api_key)
    print("WandBへのログインに成功しました。")
except Exception as e:
    print(f"WandBへのログインに失敗しました: {e}")
    print("Colabのシークレットに 'WANDB_API_KEY' が設定されているか確認してください。")

In [ ]:
# @title 2. Google Drive マウント
drive.mount('/content/drive')

In [ ]:
# @title 3. パスと設定の定義
# --- ユーザー設定エリア ---
DRIVE_ROOT = Path('/content/drive/MyDrive')
DATA_ROOT = DRIVE_ROOT / 'Archive_Wavs'
METADATA_PATH = DATA_ROOT / 'metadata.jsonl'
ZIP_DIR = DATA_ROOT

# 出力先（チェックポイント保存場所）
OUTPUT_DIR = DRIVE_ROOT / 'MusicGen_Finetuning_Output'
OUTPUT_DIR.mkdir(exist_ok=True, parents=True)

# 一時作業ディレクトリ（Colabローカル）
TEMP_WORK_DIR = Path('/content/temp_work')
TEMP_DATA_DIR = TEMP_WORK_DIR / 'data'

TEMP_DATA_DIR.mkdir(exist_ok=True, parents=True)

print(f"Metadata: {METADATA_PATH}")
print(f"Zip Dir: {ZIP_DIR}")
print(f"Output Dir: {OUTPUT_DIR}")

In [ ]:
# @title 4. ヘルパー関数の定義
def extract_zip(zip_path, extract_to):
    """ZIPファイルを指定ディレクトリに解凍する"""
    print(f"{zip_path} を {extract_to} に解凍中...")
    if extract_to.exists():
        shutil.rmtree(extract_to)
    extract_to.mkdir(parents=True, exist_ok=True)

    subprocess.run(['unzip', '-q', str(zip_path), '-d', str(extract_to)], check=True)
    print("解凍完了。")

def create_batch_metadata(main_metadata_path, current_wav_dir, output_jsonl_path):
    """
    メインのmetadata.jsonlから、現在解凍されているファイルに対応するエントリのみを抽出し、
    パスをColab上の絶対パスに書き換えて新しいjsonlを作成する。
    """
    print(f"{output_jsonl_path} にバッチメタデータを作成中...")

    extracted_files = list(current_wav_dir.rglob('*.wav'))
    extracted_files_map = {f.name: f for f in extracted_files}

    valid_entries = []

    with open(main_metadata_path, 'r', encoding='utf-8') as f:
        for line in f:
            try:
                entry = json.loads(line)
                orig_path = entry.get('path', '')
                filename = os.path.basename(orig_path)

                if filename in extracted_files_map:
                    # パスを絶対パスに更新
                    entry['path'] = str(extracted_files_map[filename])
                    # TransformersのDatasetで読み込むために 'audio' キーにパスを入れるのが一般的だが
                    # ここでは後処理でロードするため 'path' のままでもOK。
                    # ただし、datasets libraryのAudio機能を使うなら 'audio': path が便利。
                    entry['audio'] = str(extracted_files_map[filename])
                    valid_entries.append(entry)
            except json.JSONDecodeError:
                continue

    if not valid_entries:
        print("警告: 解凍されたファイルに一致するメタデータが見つかりませんでした。")
        return False

    with open(output_jsonl_path, 'w', encoding='utf-8') as f:
        for entry in valid_entries:
            f.write(json.dumps(entry) + '\n')

    print(f"{len(valid_entries)} 件のエントリを持つメタデータを作成しました。")
    return True

def preprocess_function(examples, processor, model, audio_column_name="audio", text_column_name="caption"):
    """データセットの前処理関数"""
    print("------ preprocess_function 開始 ------")

    processed_audio_arrays = []
    valid_indices = []
    # サンプリングレートを初期化（バッチ内の全サンプルで同一と仮定）
    # 最初の有効なサンプルから取得するのが安全
    sampling_rate = None

    # 音声配列のフィルタリングと処理
    for idx, audio_info in enumerate(examples[audio_column_name]):
        print(f"[Debug preprocess] Processing audio sample {idx}. Audio info: {audio_info}")
        audio_array = audio_info["array"]
        if audio_array is not None and len(audio_array) > 0:
            if sampling_rate is None:
                sampling_rate = audio_info["sampling_rate"]
                print(f"[Debug preprocess] Detected sampling rate: {sampling_rate}")

            # 必要に応じて2D（ステレオ）配列をチャンネル平均で1D（モノラル）に変換
            if audio_array.ndim == 2:
                # フォーマットは (channels, samples) または (samples, channels) と仮定
                # どちらの軸が小さい次元か（おそらくチャンネル）を確認
                if audio_array.shape[0] < audio_array.shape[1]: # (channels, samples)
                    audio_array = np.mean(audio_array, axis=0)
                else: # (samples, channels)
                    audio_array = np.mean(audio_array, axis=1)
                print(f"[Debug preprocess] Converted 2D audio to 1D. New shape: {audio_array.shape}")

            # float32型の1D NumPy配列であることを確認
            if audio_array.ndim == 1:
                processed_audio_arrays.append(audio_array.astype(np.float32))
                valid_indices.append(idx)
                print(f"[Debug preprocess] Added valid audio sample {idx}. Current processed_audio_arrays count: {len(processed_audio_arrays)}")
            else:
                print(f"バッチ内のインデックス {idx} の問題のある音声サンプルをスキップ: 変換後に1Dではありません (ndim={audio_array.ndim})。")

        else:
            print(f"バッチ内のインデックス {idx} の問題のある音声サンプルをスキップ: 空またはNoneです。")

    if not processed_audio_arrays:
        print("警告: 処理可能な音声データがこのバッチにはありません。空の入力を返します。")
        return {}

    print(f"[Debug preprocess] Number of processed audio arrays: {len(processed_audio_arrays)}")

    # 有効なインデックスに基づいてテキストをフィルタリング
    texts = [examples.get(text_column_name, [""] * len(examples[audio_column_name]))[i] for i in valid_indices]
    # プライマリキーが空または欠落している場合、他のテキストキーを処理
    for i, text_val in enumerate(texts):
        if not text_val:
            if "text" in examples and valid_indices[i] < len(examples["text"]):
                texts[i] = examples["text"][valid_indices[i]]
            elif "description" in examples and valid_indices[i] < len(examples["description"]):
                texts[i] = examples["description"][valid_indices[i]]
    # すべてのテキストが文字列であることを確認
    texts = [str(t) if t else "" for t in texts]
    print(f"[Debug preprocess] Texts for batch: {texts[:5]}...") # Print first 5 texts

    # 音声とテキストの最大長を定義
    # MusicGenは通常約30秒の音声を処理 (32kHz * 30s = 960,000 samples)
    # Encodecの一般的な最大長を使用 (約7.68秒 = 245760 samples)
    MAX_AUDIO_SAMPLES = 245760 # Approx 7.68 seconds at 32kHz
    MAX_TEXT_TOKENS = 256

    # 特徴抽出器を使用して音声入力を処理
    audio_features = processor.feature_extractor(
        processed_audio_arrays, # Use cleaned audio arrays
        sampling_rate=sampling_rate,
        padding="max_length",
        max_length=MAX_AUDIO_SAMPLES,
    )
    print(f"[Debug preprocess] audio_features.input_values type: {type(audio_features.input_values)}")

    # テンソル変換前に input_values と attention_mask のNumPy配列の一貫性を手動で確認
    input_values_to_normalize = audio_features.input_values
    if not isinstance(input_values_to_normalize, list):
        input_values_to_normalize = [input_values_to_normalize]

    normalized_input_values = []
    for audio_arr in input_values_to_normalize:
        if not isinstance(audio_arr, np.ndarray):
            print(f"input_values 内の不正な形式の audio_arr をスキップ: type={type(audio_arr)}")
            continue

        if audio_arr.ndim == 2:
            if audio_arr.shape[0] < audio_arr.shape[1]:
                audio_arr = np.mean(audio_arr, axis=0)
            else:
                audio_arr = np.mean(audio_arr, axis=1)

        if audio_arr.ndim != 1:
            print(f"feature_extractor からの問題のある音声サンプルをスキップ: 変換後に1Dではありません (ndim={audio_arr.ndim})。")
            continue

        current_length = audio_arr.shape[-1]
        if current_length > MAX_AUDIO_SAMPLES:
            normalized_input_values.append(audio_arr[:MAX_AUDIO_SAMPLES])
        elif current_length < MAX_AUDIO_SAMPLES:
            normalized_input_values.append(np.pad(audio_arr, (0, MAX_AUDIO_SAMPLES - current_length), 'constant', constant_values=0))
        else:
            normalized_input_values.append(audio_arr)

    if not normalized_input_values:
        print("警告: 処理可能な正規化された音声入力がこのバッチにはありません。空の入力を返します。")
        return {}

    input_values_np = np.stack(normalized_input_values).astype(np.float32)
    input_values_np = input_values_np[:, np.newaxis, :]

    print(f"[Debug preprocess] input_values_np final shape: {input_values_np.shape}, dtype: {input_values_np.dtype}")

    audio_inputs_tensor = {
        "input_values": torch.from_numpy(input_values_np),
    }

    if "attention_mask" in audio_features and audio_features.attention_mask is not None:
        attention_mask_to_normalize = audio_features.attention_mask
        if not isinstance(attention_mask_to_normalize, list):
            attention_mask_to_normalize = [attention_mask_to_normalize]

        normalized_attention_mask = []
        for mask_arr in attention_mask_to_normalize:
            if not isinstance(mask_arr, np.ndarray):
                print(f"attention_mask 内の不正な形式の mask_arr をスキップ: type={type(mask_arr)}")
                continue

            if mask_arr.ndim == 2:
                if mask_arr.shape[0] < mask_arr.shape[1]:
                    mask_arr = np.mean(mask_arr, axis=0).astype(np.longlong)
                else:
                    mask_arr = np.mean(mask_arr, axis=1).astype(np.longlong)

            if mask_arr.ndim != 1:
                print(f"feature_extractor からの問題のある mask_arr をスキップ: 変換後に1Dではありません (ndim={mask_arr.ndim})。")
                continue

            current_length = mask_arr.shape[-1]
            if current_length > MAX_AUDIO_SAMPLES:
                normalized_attention_mask.append(mask_arr[:MAX_AUDIO_SAMPLES])
            elif current_length < MAX_AUDIO_SAMPLES:
                normalized_attention_mask.append(np.pad(mask_arr, (0, MAX_AUDIO_SAMPLES - current_length), 'constant', constant_values=0))
            else:
                normalized_attention_mask.append(mask_arr)

        if not normalized_attention_mask:
            audio_inputs_tensor["attention_mask"] = torch.ones(
                audio_inputs_tensor["input_values"].shape[0],
                audio_inputs_tensor["input_values"].shape[2],
                dtype=torch.long
            )
        else:
            attention_mask_np = np.stack(normalized_attention_mask).astype(np.longlong)
            audio_inputs_tensor["attention_mask"] = torch.from_numpy(attention_mask_np)
    else:
        audio_inputs_tensor["attention_mask"] = torch.ones(
            audio_inputs_tensor["input_values"].shape[0],
            audio_inputs_tensor["input_values"].shape[2],
            dtype=torch.long
        )
    print(f"[Debug preprocess] audio_inputs_tensor['attention_mask'] shape: {audio_inputs_tensor['attention_mask'].shape}, dtype: {audio_inputs_tensor['attention_mask'].dtype}")

    text_inputs = processor.tokenizer(
        text=texts,
        padding="max_length",
        truncation=True,
        max_length=MAX_TEXT_TOKENS,
        return_tensors="pt",
    )
    print(f"[Debug preprocess] text_inputs.input_ids shape: {text_inputs.input_ids.shape}, dtype: {text_inputs.input_ids.dtype}")

    with torch.no_grad():
        input_values_for_encode = audio_inputs_tensor["input_values"].to(model.device)
        padding_mask_for_encode = audio_inputs_tensor["attention_mask"].to(model.device)

        print(f"[Debug preprocess] Before audio_encoder.encode - input_values_for_encode shape: {input_values_for_encode.shape}, dtype: {input_values_for_encode.dtype}, device: {input_values_for_encode.device}")
        print(f"[Debug preprocess] Before audio_encoder.encode - padding_mask_for_encode shape: {padding_mask_for_encode.shape}, dtype: {padding_mask_for_encode.dtype}, device: {padding_mask_for_encode.device}")

        if not torch.isfinite(input_values_for_encode).all():
            print(f"警告: input_values_for_encode に非有限値 (NaN/Inf) が含まれています。バッチをスキップします。")
            return {}

        audio_codes = None
        try:
            with torch.amp.autocast(device_type="cuda", dtype=torch.float16):
                audio_codes, audio_scales, _ = model.audio_encoder.encode(
                    input_values_for_encode,
                    padding_mask_for_encode,
                    model.audio_encoder.config.target_bandwidths[-1]
                )
        except Exception as e:
            print(f"エラー: model.audio_encoder.encode 中に例外が発生しました: {e}")
            print("------ preprocess_function 終了 (エラー) ------")
            return {}

        if not isinstance(audio_codes, torch.Tensor):
            print(f"エラー: model.audio_encoder.encode が予期しない非テンソル型の audio_codes を返しました: {type(audio_codes)}")
            print(f"詳細: audio_codes の値は '{str(audio_codes)}' でした。")
            print("警告: オーディオエンコーダーからの予期せぬ出力のため、このバッチのデータ処理をスキップします。")
            print("------ preprocess_function 終了 (エラー) ------")
            return {}

        print(f"[Debug preprocess] After audio_encoder.encode - audio_codes shape: {audio_codes.shape}, dtype: {audio_codes.dtype}")

        if audio_codes.shape[-1] > 1:
            labels_audio_codes = audio_codes[:, 1:]
            decoder_input_ids_audio_codes = audio_codes[:, :-1]
        else:
            print("警告: オーディオコードが短すぎてシフトできません。labels と decoder_input_ids を同じ値に設定します。")
            labels_audio_codes = audio_codes
            decoder_input_ids_audio_codes = audio_codes

        labels_audio_codes = labels_audio_codes.cpu()
        decoder_input_ids_audio_codes = decoder_input_ids_audio_codes.cpu()

    inputs = {
        "input_values": audio_inputs_tensor["input_values"],
        "padding_mask": audio_inputs_tensor["attention_mask"],
        "input_ids": text_inputs.input_ids,
        "attention_mask": text_inputs.attention_mask,
        "decoder_input_ids": decoder_input_ids_audio_codes,
        "labels": labels_audio_codes,
    }
    print("------ preprocess_function 終了 (成功) ------")
    return inputs

In [ ]:
# @title 5. メインループ実行
import torch # Added import statement for torch
from transformers import AutoProcessor, BitsAndBytesConfig, MusicgenForConditionalGeneration
from peft import LoraConfig, get_peft_model # Added LoraConfig and get_peft_model imports

# モデルとプロセッサの準備
MODEL_ID = "facebook/musicgen-large"
print(f"モデルをロード中: {MODEL_ID}...")

# 8-bit 量子化設定
quantization_config = BitsAndBytesConfig(
    load_in_8bit=True,
    bnb_8bit_compute_dtype=torch.float16 # A100/V100ならfloat16を推奨
)

processor = AutoProcessor.from_pretrained(MODEL_ID)
model = MusicgenForConditionalGeneration.from_pretrained(
    MODEL_ID,
    quantization_config=quantization_config, # 8-bit 量子化を適用
    device_map="auto" # GPUで実行するように変更
)

# PEFT (LoRA) の設定
# MusicGenのエンコーダ (T5EncoderModel) とデコーダ (MusicgenForCausalLM) の両方にLoRAを適用
lora_config = LoraConfig(
    r=8,  # LoRAのランク
    lora_alpha=32, # LoRAスケーリング係数
    target_modules=["q_proj", "v_proj"], # LoRAを適用するモジュール (Attention層のQuery, Value)
    lora_dropout=0.05,
    bias="none",
    task_type="SEQ_2_SEQ_LM", # MusicGenはSequence-to-Sequenceモデル
)

# PEFTモデルをオリジナルモデルにアタッチ
model = get_peft_model(model, lora_config)
model.print_trainable_parameters() # 学習可能なパラメータ数を確認

model.train()

# ZIPファイルリスト取得
zip_files = sorted(list(ZIP_DIR.glob('archive_batch_*.zip')))
print(f"{len(zip_files)} 個のZIPファイルが見つかりました。")

# 以前のチェックポイントがあればロード（簡易実装）
latest_checkpoint_path = OUTPUT_DIR / 'latest_checkpoint'
if latest_checkpoint_path.exists():
    print(f"{latest_checkpoint_path} から再開します...")
    # PEFTモデルとしてロードする場合
    model = MusicgenForConditionalGeneration.from_pretrained(
        latest_checkpoint_path,
        quantization_config=quantization_config,
        device_map="auto" # GPUで実行するように変更
    )
    # LoRAアダプターをロード
    model = get_peft_model(model, lora_config)

# GPU設定 (device_map="auto"を使用しているため、model.to(device)は不要)
device = "cuda" if torch.cuda.is_available() else "cpu"

# CUDAキャッシュをクリアしてメモリを解放
torch.cuda.empty_cache()

for i, zip_file in enumerate(zip_files):
    print(f"\n{'='*40}")
    print(f"バッチ {i+1}/{len(zip_files)} を処理中: {zip_file.name}")
    print(f"{'='*40}")

    # 1. 解凍
    extract_zip(zip_file, TEMP_DATA_DIR)

    # 2. メタデータ作成
    batch_metadata_path = TEMP_WORK_DIR / 'batch.jsonl'
    success = create_batch_metadata(METADATA_PATH, TEMP_DATA_DIR, batch_metadata_path)

    if not success:
        print("メタデータエラーのため、このバッチをスキップします。")
        continue

    # 3. データセット準備
    dataset = load_dataset("json", data_files=str(batch_metadata_path), split="train")
    dataset = dataset.cast_column("audio", Audio(sampling_rate=processor.feature_extractor.sampling_rate))

    # 前処理の適用
    print("データセットを前処理中...")
    encoded_dataset = dataset.map(
        lambda x: preprocess_function(x, processor, model),
        batched=True,
        remove_columns=dataset.column_names,
        batch_size=4 # メモリに応じて調整
    )

    # 4. トレーニング設定
    # バッチごとにTrainerを作り直すが、modelは同じオブジェクトを使い回すことで学習を継続する
    training_args = TrainingArguments(
        output_dir=str(TEMP_WORK_DIR / "results"),
        per_device_train_batch_size=2, # A100ならもう少し増やせるかも
        gradient_accumulation_steps=4,
        learning_rate=1e-5,
        num_train_epochs=5, # 1バッチあたりのエポック数
        save_steps=1000, # バッチ内での保存頻度（必要なら）
        logging_steps=10,
        fp16=True, # GPU実行のためTrueに設定
        save_total_limit=1,
        remove_unused_columns=False,
        dataloader_num_workers=2,
        report_to="wandb", # WandB有効化
        run_name=f"musicgen-finetuning-batch-{i+1}", # バッチごとにRun名を分ける
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=encoded_dataset,
    )

    print("このバッチのトレーニングを開始します...")
    trainer.train()

    # 5. モデル保存
    # バッチ完了ごとにDriveへ保存
    save_path = OUTPUT_DIR / f'checkpoint_batch_{i+1}'
    print(f"モデルを {save_path} に保存中...")
    model.save_pretrained(save_path)
    processor.save_pretrained(save_path)

    # 最新版として上書き
    latest_path = OUTPUT_DIR / 'latest_checkpoint'
    model.save_pretrained(latest_path)
    processor.save_pretrained(latest_path)

    # 6. クリーンアップ
    print("一時データをクリーンアップ中...")
    shutil.rmtree(TEMP_DATA_DIR)
    TEMP_DATA_DIR.mkdir(exist_ok=True)
    # Trainerのクリーンアップ（メモリ解放のため）
    del trainer
    del dataset
    del encoded_dataset
    torch.cuda.empty_cache()

print("すべてのバッチが処理されました。")